## Introduction

Before doing anything, let's try to get a GPU instance!

TO DOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

This first cell needs to be executed to set up the environment and instantiate the client used to connect to a proof assistant.

In [29]:
# %pip install ollama
# %pip install colab-xterm

# !sudo apt-get update
# !sudo apt-get install pciutils lshw

# !curl -fsSL https://ollama.com/install.sh | sh
# url = "https://raw.githubusercontent.com/theostos/small-pytanque-tp/refs/heads/main/client.py"
# !wget --no-cache --backups=1 {url}
# url = "https://raw.githubusercontent.com/theostos/small-pytanque-tp/refs/heads/main/utils.py"
# !wget --no-cache --backups=1 {url}
# !pip install requests

In [54]:
from client import ProofAssistantClientAPI, goals_to_str

client = ProofAssistantClientAPI("http://128.93.101.129:8765")

On the server side, there is a file containing some theorems. First, let's look at them.

In [55]:
for section in client.sections():
  print(f"Theorems in section: {section}")
  for k in range(client.num_thm(section)):
    thm = client.show_thm(section, k)
    print(f"Theorem {k}, {thm['name']} description: {thm['statement']}")
  print()


Theorems in section: introduction
Theorem 0, involution_injective description: forall x y, f x = f y -> x = y

Theorems in section: logic
Theorem 0, Modus_ponens description: forall P Q : Prop, P -> (P -> Q) -> Q
Theorem 1, Modus_tollens description: forall P Q : Prop, ~Q /\ (P -> Q) -> ~P
Theorem 2, Disjunctive_syllogism description: forall P Q : Prop, (P \/ Q) -> ~P -> Q
Theorem 3, DeMorgan1 description: forall P Q : Prop, ~P \/ ~Q -> ~(P /\ Q)
Theorem 4, DeMorgan2 description: forall P Q : Prop, ~P /\ ~Q -> ~(P \/ Q)
Theorem 5, DeMorgan3 description: forall P Q : Prop, ~(P \/ Q) -> ~P /\ ~Q
Theorem 6, NotNot_LEM description: forall P : Prop, ~ ~(P \/ ~P)

Theorems in section: math
Theorem 0, unique_max description: (A : R -> Prop) (x y : R) : x is_a_max_of A -> y is_a_max_of A -> x = y
Theorem 1, inf_lt description: (A : R -> Prop) (x : R) : x is_an_inf_of A -> forall y, x < y -> exists a, A a /\ a < y
Theorem 2, le_of_le_add_eps description: (x y : R) : (forall eps, eps > 0 -> y <=

### A case study: Lelarge's Theorem

We will start by proving the Lelarge's Theorem in the Introduction section.
Let's review it by selecting the first theorem (index = 0) in the "introduction" section.

Before doing so, we need to examine the current context (i.e. the available lemmas, definitions, and notations) to understand it.

In [ ]:
thm =  client.show_thm("introduction", 0)
print("Context:")
print("\n".join(thm['premises']))
print("\n")
print(f"Theorem {thm['name']}: {thm['statement']}",)

Theorem involution_injective states that an arbitrary function f, satisfying Hinv (i.e. being an involution), is one-to-one.

To prove it, we will use the following tactics (read from left to right):

* "intros x y H." introduces two variables, x and y, associated with the forall quantifier, and the hypothesis H corresponding to f x = f y.
* "rewrite <- Hinv with (x := x)." rewrites the left-hand side of the current goal using the hypothesis specialized with x = x.
* "rewrite <- Hinv with (x := y)." rewrites the left-hand side of the current goal using the hypothesis specialized with x = y.
* "rewrite H." rewrites the current goal using the introduced hypothesis H (i.e. f x = f y).
* "reflexivity." discharges goals of the form a = a.

In [ ]:
state, goals = client.start_thm("introduction", 0)
print("Started theorem session:")
print(goals_to_str(goals))

In [ ]:
state, goals = client.run_tac(state, "intros x y H.")
print(goals_to_str(goals))

In [ ]:
state, goals = client.run_tac(state, "rewrite <- Hinv with (x := x).")
print(goals_to_str(goals))

In [ ]:
state, goals = client.run_tac(state, "rewrite <- Hinv with (x := y).")
print(goals_to_str(goals))

In [ ]:
state, goals = client.run_tac(state, "rewrite H.")
print(goals_to_str(goals))

In [ ]:
state, goals = client.run_tac(state, "reflexivity.")
print(goals_to_str(goals))

# Automation

## Sledge Hammer

Let's try to automate theorem proving. In deep learning our sledge hammer is big LLM, and one of them is chatGPT.
Is it able to prove Lelarge's theorem?

First, let's ask it to one of the best "reasoning" model: GPT o3-mini.
Extract the sequence of tactics from [this](https://chatgpt.com/share/67e9842e-7d18-8007-b394-d29b03d859cb) link, and try to submit it to the Rocq server ([petanque server](https://github.com/ejgallego/coq-lsp/tree/main/petanque)).

To do it, complete the following cell. You only need to add the remaining tactics

In [ ]:
# complete the following list with the sequence of tactics provided by GPT o3-mini
tactics = ['intros x y H.', 'TO COMPLETE: SECOND TACTIC', 'AND SO ON']

state, goals = client.start_thm("introduction", 0)
for tactic in tactics:
    state, goals = client.run_tac(state, tactic)
    print(goals_to_str(goals))

Remarkably, if we ask more modest model things get a bit more complex:
* Asking to GPT 4o model (see [here](https://chatgpt.com/share/67e986d3-6650-8007-b01c-b5dfe48468a5)), it is able to prove it after 5 attempts.
* Asking to GPT 4o-mini (see [here](https://chatgpt.com/share/67e9871d-9de8-8007-aaeb-04d48d61b525)) is not able to prove it in 8 attempts.

And even more remarkably, both non reasoning and reasoning model from [DeepSeek](https://chat.deepseek.com/) are able to solve this problem at the first attempt.

## Open-weight model

Now, let's try to do it with an open-weight locally run model. We choose Gemma 3 12b, since it has good performance while being able to run on a collab T4 instance.

First we need to start an inference engine (to serve our model locally).

In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"])

Then, let's download our model.

In [ ]:
!ollama pull hf.co/unsloth/gemma-3-27b-it-GGUF:Q4_K_M

In [ ]:
import ollama

def get_response(prompt):
  response = ollama.chat(
        model="unsloth/gemma-3-27b-it-GGUF:Q4_K_M",
        messages=[
            {"role": "user", "content": prompt},
        ],
        options={"temperature":0.6}
    )
  return response["message"]["content"]

In [ ]:
import re
from collections import defaultdict

def parse_output(output):
    # to avoid some parsing issue, we accept instruction to not end with a point as normally required.
    pattern = r'```coq\n(.*?)\.?\n'
    match_output = re.search(pattern, output)
    if match_output:
      output = match_output.group(1).strip()
      return output + '.'
    return ''


prompt_template = """You are an expert in Coq, a theorem-proving assistant. Your task is to help progress a formal proof by providing exactly one correct and effective Coq tactic to advance towards the goal.

Here is a brief explanation of tactics you may use:

- intros: Introduces hypotheses or variables into the context. (example: "intros P Q." introduces hypotheses P and Q.)
- apply: Applies a hypothesis or theorem to match the current goal. (example: "apply H0." If the goal is Q and you have a hypothesis H0: P -> Q, applying H0 changes the goal to P.)
- exact: Directly solves the current goal if you have an exact matching hypothesis. (example: "exact H." If the goal is P and you have a hypothesis H: P, then the goal is resolved.)
- contradiction: Resolves the goal if there is a contradiction in the hypotheses. (example: if you have hypotheses H1: P -> False and H2: P, using "contradiction." resolves the goal.)
- unfold not: Expands the definition of negation (~P becomes P -> False). (examples: "unfold not in H." applies it to hypothesis H, "unfold not." applies it to the goal.)
- inversion: Breaks apart hypotheses involving conjunctions (and), disjunctions (or), or existential quantifiers to reveal simpler components. (example: "inversion H." breaks hypothesis H into simpler parts.)
- split: Splits goals involving conjunctions into separate subgoals. (example: "split." transforms goal P /\ Q into two separate goals, P and Q.)
- left/right: Selects a side of a disjunction (or) goal to prove. (examples: "left." to prove the left side of a goal P \/ Q, "right." to prove the right side.)

Current proof state:
{goal}

Carefully analyze the current goal, consider available hypotheses, and propose the most logical and efficient next step in the proof.

Respond with ONLY ONE Coq tactic enclosed in a Coq code block. Ensure the tactic is syntactically correct and directly applicable. Don't write any comment, simple the code block.

Example of correct formatting:
```coq
tactic_1.
```

"""

prompt_remove = """Don't use any of the following instructions:
{remove}
"""
thm =  client.show_thm("logic", 5)
statement = f"Theorem {thm['name']}: {thm['statement']}"
premises = "\n".join(thm['premises'])

success = False
for _ in range(1):
  state, goals = client.start_thm("logic", 5)
  print("New try")
  tactics = []
  useless_tactics = defaultdict(list)
  failed_tactics = []
  for _ in range(100):
    to_prove_pp = goals_to_str(goals)
    to_prove = to_prove_pp.replace('|-', 'to prove: ')

    prompt = prompt_template.format(goal=to_prove)
    if failed_tactics:
      prompt += prompt_remove.format(remove="\n".join(failed_tactics))
    
    if useless_tactics[to_prove_pp]:
      prompt += prompt_remove.format(remove="\n".join(useless_tactics[to_prove_pp]))
  
    output = get_response(prompt)
    next_tactic = parse_output(output)
    try:
      state, goals = client.run_tac(state, next_tactic)
      new_to_prove_pp = goals_to_str(goals)
      if to_prove_pp == new_to_prove_pp:
        useless_tactics[new_to_prove_pp].append(next_tactic)
      print(next_tactic)
      tactics.append(next_tactic)
      failed_tactics = []
    except Exception as e:
      failed_tactics.append(next_tactic)
      pass
    if not goals:
      print("Finished!")
      success = True
      break
    
  if success:
    print("Found solution:\n" + "\n".join(tactics))
    break
  else:
    print("Failed")
    
  

Now, let's try to find better prompts!

In [ ]:
prompt_bis = """You are an expert in Coq, a theorem-proving assistant. Your task is to help progress a formal proof involving real numbers, upper/lower bounds, limits, and other related concepts by providing exactly one correct and effective Coq tactic to advance towards the goal.

Here is a brief explanation of tactics you may use:

- intros: Introduces hypotheses or variables into the context. (example: "intros x y." introduces hypotheses x and y.)
- apply: Applies a hypothesis or theorem to match the current goal. (example: "apply H." If the goal is Q and you have a hypothesis H: P -> Q, applying H changes the goal to P.)
- exact: Directly solves the current goal if you have an exact matching hypothesis. (example: "exact H." If the goal is P and you have a hypothesis H: P, then the goal is resolved.)
- contradiction: Resolves the goal if there is a contradiction in the hypotheses. (example: if you have hypotheses H1: P -> False and H2: P, using "contradiction." resolves the goal.)
- unfold: Expands definitions. (examples: "unfold is_maximum in H." expands the definition of is_maximum in hypothesis H, "unfold limit." expands the definition of limit in the goal.)
- inversion: Breaks apart hypotheses involving conjunctions (and), disjunctions (or), or existential quantifiers to reveal simpler components. (example: "inversion H." breaks hypothesis H into simpler parts.)
- split: Splits goals involving conjunctions into separate subgoals. (example: "split." transforms goal P /\ Q into two separate goals, P and Q.)
- destruct: Breaks a hypothesis or goal involving existential quantifiers or logical connectives into simpler components. (example: "destruct H as [a Ha]." if H is "exists a, P a", creating hypothesis Ha: P a.)
- left/right: Selects a side of a disjunction (or) goal to prove. (examples: "left." to prove the left side of a goal P \/ Q, "right." to prove the right side.)
- specialize: Instantiates hypotheses with specific values or assumptions. (example: "specialize (H eps)." instantiates hypothesis H with eps.)
- assert: Introduces an intermediate statement to help advance the proof. (example: "assert (H : P)." creates a new subgoal to prove P.)
- lra: Automatically solves linear inequalities involving real numbers.
- lia: Automatically solves linear inequalities involving integers.

Current proof state:
{goal}

Carefully analyze the current goal, consider available hypotheses, and propose the most logical and efficient next step in the proof.

Respond with ONLY ONE Coq tactic enclosed in a Coq code block. Ensure the tactic is syntactically correct and directly applicable. Don't write any comment, simply the code block.

Example of correct formatting:
```coq
tactic_1.
```
"""